In [3]:
import gseapy as gp
import pandas as pd
import os

In [4]:
gene_sets = {}
for file in ["../Homo_sapiens_HALLMARK_OXIDATIVE_PHOSPHORYLATION.csv", 
             "../Homo_sapiens_KEGG_CITRATE_CYCLE_TCA_CYCLE.csv", 
             "../Homo_sapiens_REACTOME_MITOCHONDRIAL_BIOGENESIS.csv"]:
    df = pd.read_csv(file, header=0, index_col=0)
    set = os.path.basename(file).split(".")[0].replace("Homo_sapiens_", "")
    gene_sets[set] = df["gene_symbol"].tolist()

FileNotFoundError: [Errno 2] No such file or directory: '../Homo_sapiens_HALLMARK_OXIDATIVE_PHOSPHORYLATION.csv'

In [8]:
rnk_df = pd.read_csv("../ranking_expression.rnk", sep="\t", header=None)
print(rnk_df.head())
print(rnk_df.dtypes)

          0          1
0   MT-RNR2  43.280851
1      HULC  14.520045
2   MT-RNR1  14.212717
3      GPX1  13.967892
4  MTATP6P1  13.106793
0     object
1    float64
dtype: object


In [11]:
pre_res = gp.prerank(rnk=rnk_df,
                    gene_sets=gene_sets)

In [3]:
phenoA, phenoB, class_vector =  gp.parser.gsea_cls_parser("/Users/Macbook/Documents/rnaseq_sandbox/annotations_expression.cls")
print(class_vector)
gene_exp = pd.read_csv("../deseq2_norm_expression.gct", sep="\t", header=2)
gene_exp.head()

['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'High', 'High', 'High', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low']


,Name,Description,norm1_Low,norm3_Low,norm4_Low,norm5_Low,cirr1_Low,cirr2_Low,cirr3_Low,cirr4_Low,...,P9_Low,P13_Low,P14_Low,H02_Low,H13_Low,H16_Low,H18_Low,H22_Low,H25_Low,zhao9_Low
0,A1BG,ENSG00000121410,385.624594,262.674098,513.800426,427.173163,414.279154,478.374807,540.956438,460.942106,...,65.771813,4.094317,0.447774,45.644224,0.000000,96.413566,0.000000,260.625530,116.915907,0.000000
1,A1BG-AS1,ENSG00000268895,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,15.214741,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,A1CF,ENSG00000148584,34.573239,52.084092,25.690021,22.019235,34.871280,26.678595,15.977616,0.000000,...,44.555099,28.660218,38.465203,45.644224,57.230384,144.620348,192.996168,81.445478,58.457953,33.556357
3,A2M,ENSG00000175899,162.228277,117.690015,42.816702,130.647462,195.028594,371.353776,255.641861,48.520222,...,26.308725,4.094317,15.729856,76.073707,0.000000,0.000000,0.000000,100.992393,11.691591,0.000000
4,A2M-AS1,ENSG00000245105,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.173332,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [4]:
print("positively correlated: ", phenoA)
print("negtively correlated: ", phenoB)

positively correlated:  Low
negtively correlated:  High


In [12]:
gs_res = gp.gsea(data=gene_exp,
         gene_sets=gene_sets, 
         cls = class_vector,
         # set permutation_type to phenotype if samples >=15
         permutation_type='phenotype', 
         permutation_num=100000, # reduce number to speed up test
         outdir="sob2", 
         method='log2_ratio_of_classes')
gs_res.pheno_pos = "Low"
gs_res.pheno_neg = "High"
gs_res.run()

2025-07-28 15:32:36,482 Warning: Input data contains NA, filled NA with 0
2025-07-28 15:32:41,118 Warning: Input data contains NA, filled NA with 0


In [5]:
degs = pd.read_csv("/Users/Macbook/Documents/rnaseq_sandbox/leiden_ranking.csv")#.set_index("Unnamed: 0")
degs

,Unnamed: 0,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,A1BG,168.486472,0.385659,1.006529,0.383158,0.701603,0.877587
1,A1BG-AS1,0.448720,3.183535,2.608592,1.220403,0.222312,NaN
2,A1CF,47.163183,-0.174416,0.349216,-0.499451,0.617462,0.829894
3,A2M,110.821895,-0.295119,0.892997,-0.330481,0.741037,0.893053
4,A2M-AS1,0.085591,0.432226,1.308095,0.330424,0.741080,NaN
...,...,...,...,...,...,...,...
20434,ZNF426-DT,0.001783,3.024092,4.144844,0.729603,0.465633,NaN
20435,ZNF451-AS1,0.001337,2.983865,3.481728,0.857007,0.391441,NaN
20436,ZNF460-AS1,0.002921,3.007617,3.704095,0.811971,0.416808,NaN
20437,GSTT1,0.422708,3.242297,4.565075,0.710240,0.477556,NaN


In [7]:
pre_res = gp.prerank(degs.loc[:,['Unnamed: 0', 'log2FoldChange']], gene_sets=gene_sets)

2025-07-29 09:53:16,158 No gene sets passed through filtering condition!!!, try new parameters again!
Note: check gene name, gmt file format, or filtering size.


SystemExit: 0

/opt/anaconda3/envs/scrna_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
degs.loc[:, ["Unnamed: 0", "log2FoldChange"]].to_csv("ranking_leiden.csv", index=False, sep = "\t")